In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [2]:
url_inicial = 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/archive'
url_root = 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/archive'
r=requests.get(url_inicial)

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('div',class_='issue-summary media')
vol = [x.find('a').get('href')for x in volumen]
vol

['https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/686',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/658',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/634',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/608',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/577',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/554',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/17',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/16',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/15',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/14',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/view/13',
 'https://revistas.juridicas.unam.mx/index.php/derecho-inte

In [5]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='media-list') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [6]:
vol2

['https://revistas.juridicas.unam.mx/index.php/derecho-internacional/article/view/16980',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/article/view/15624',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/article/view/14497',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/article/view/13317',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/article/view/13318',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/article/view/13319',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/article/view/13320',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/article/view/13321',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/article/view/13322',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/article/view/13323',
 'https://revistas.juridicas.unam.mx/index.php/derecho-internacional/article/view/13324',
 'https://

In [7]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='media-list') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [8]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://revistas.juridicas.unam.mx/index.php/derecho-internacional/issue/archive


In [9]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

98

In [10]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [11]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Anuario Mexicano de Derecho Internacional'
    a='V. Ciencias Sociales'
    tem='Derecho Internacional'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page-header').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='article-abstract').find('p').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link btn btn-primary pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [12]:
list_scraper=list_scraper[0:98]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [13]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Revista completa,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
1,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Revista completa,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
2,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Revista completa,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
3,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Amnistías y derecho internacional. El conflict...,Las consecuencias propias de la concesión de a...,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
4,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,¿Una nueva guerra fría por otros medios? Las d...,Este trabajo analiza las expulsiones recíproca...,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
...,...,...,...,...,...,...,...,...
93,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Premilinaries,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
94,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Preliminares,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
95,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Preliminares,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
96,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Preliminares,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...


In [15]:
df_catalogo.to_csv('Revista25.csv')

In [16]:
df=df_catalogo.drop(df_catalogo.index[[0,1,2,51,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89]])
df

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
3,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Amnistías y derecho internacional. El conflict...,Las consecuencias propias de la concesión de a...,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
4,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,¿Una nueva guerra fría por otros medios? Las d...,Este trabajo analiza las expulsiones recíproca...,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
5,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,La imprecisa regulación jurídica de los proces...,La secesión postcolonial es uno de los temas m...,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
6,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,El desafío soberanista y el derecho internacional,Vamos a analizar la intención de las autoridad...,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
7,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,The Role of Non-Governmental Organizations in ...,Las organizaciones no gubernamentales han dese...,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
...,...,...,...,...,...,...,...,...
93,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Premilinaries,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
94,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Preliminares,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
95,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Preliminares,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
96,Anuario Mexicano de Derecho Internacional,V. Ciencias Sociales,Derecho Internacional,Preliminares,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...


In [17]:
df.to_csv('Revista25.csv', index=False)